In [1]:
from ultralytics import YOLO
import cv2

# Загрузка модели YOLOv10
model = YOLO("yolov10n.pt") 

# Укажите путь к вашему видеофайлу
video_path = "videoAUD.mp4"  # Замените на ваш файл (mp4, avi и т.д.)

# Инициализация видеопотока
cap = cv2.VideoCapture(video_path)

# Проверка успешного открытия файла
if not cap.isOpened():
    print(f"Ошибка: Не удалось открыть видеофайл {video_path}")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Выход при окончании видео

    # Детекция людей (класс 0)
    results = model(frame, classes=[0], verbose=False)

    # Визуализация результатов
    annotated_frame = results[0].plot()
    cv2.imshow("YOLOv10 Video Analysis", annotated_frame)

    # Выход по нажатию 'q'
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
from ultralytics import YOLO
import cv2

# Загрузка модели YOLOv10
model = YOLO("yolov8n.pt") 

# Путь к видеофайлу
video_path = "videoAUD.mp4"

# Инициализация видеопотока
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Ошибка: Не удалось открыть видеофайл {video_path}")
    exit()

# Получение размеров кадра
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Определение зон с хранением точек
zones = [
    {'name': 'Left Zone', 'coords': (0, 0, width//3, height), 'count': 0, 'points': []},
    {'name': 'Center Zone', 'coords': (width//3, 0, 2*width//3, height), 'count': 0, 'points': []},
    {'name': 'Right Zone', 'coords': (2*width//3, 0, width, height), 'count': 0, 'points': []},
]

# Цвета для зон и точек
zone_colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0)]

# Настройки вывода информации
text_position = (20, 50)  # Начальная позиция текста
line_height = 40  # Высота строки
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_thickness = 2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Сброс счетчиков и точек для текущего кадра
    for zone in zones:
        zone['count'] = 0
        zone['points'] = []

    # Детекция людей
    results = model(frame, classes=[0], verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy()

    # Подсчет людей и сохранение точек
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        x_center = (x1 + x2) // 2
        y_center = (y1 + y2) // 2

        # Проверка принадлежности зоне
        for zone in zones:
            zx1, zy1, zx2, zy2 = zone['coords']
            if zx1 <= x_center <= zx2 and zy1 <= y_center <= zy2:
                zone['count'] += 1
                zone['points'].append((x_center, y_center))
                break

    # Отрисовка зон, точек и счетчиков
    annotated_frame = frame.copy()
    
    # 1. Рисуем зоны
    for i, zone in enumerate(zones):
        color = zone_colors[i]
        zx1, zy1, zx2, zy2 = zone['coords']
        
        # Рисуем прямоугольник зоны
        cv2.rectangle(annotated_frame, (zx1, zy1), (zx2, zy2), color, 2)
        
        # Рисуем точки в зоне
        for (x, y) in zone['points']:
            cv2.circle(annotated_frame, (x, y), radius=5, color=color, thickness=-1)

    # 2. Рисуем детекции поверх зон
    annotated_frame = results[0].plot(img=annotated_frame)

    # 3. Добавляем текстовые метки счетчиков
    # Создаем копию для наложения полупрозрачного фона
    overlay = annotated_frame.copy()
    cv2.rectangle(overlay, 
                 (text_position[0]-10, text_position[1]-35), 
                 (text_position[0] + 250, text_position[1] + line_height*len(zones)), 
                 (0, 0, 0), -1)
    alpha = 0.6  # Прозрачность фона
    annotated_frame = cv2.addWeighted(overlay, alpha, annotated_frame, 1 - alpha, 0)

    # Выводим счетчики
    for i, zone in enumerate(zones):
        text = f"{zone['name']}: {zone['count']}"
        color = zone_colors[i]
        y_pos = text_position[1] + i*line_height
        cv2.putText(annotated_frame, text, 
                   (text_position[0], y_pos),
                   font, font_scale, color, font_thickness, cv2.LINE_AA)

    # Отображение результата
    cv2.imshow("YOLOv8 Video Analysis", annotated_frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
from ultralytics import YOLO
import cv2
import psycopg2
from datetime import datetime
from flask import Flask, render_template, request, jsonify, send_file
import pandas as pd
import threading
import io
import logging

# Настройка логирования
logging.basicConfig(level=logging.DEBUG)

# Инициализация Flask
app = Flask(__name__)

# Конфигурация PostgreSQL
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'dbname': 'people_counter',
    'user': 'pc_user',
    'password': 'strong_password'
}

# Инициализация модели YOLO
model = YOLO("yolov8n.pt")
video_path = "videoAUD.mp4"

# Инициализация видеопотока
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# Определение зон
zones = [
    {'name': 'Left Zone', 'coords': (0, 0, width//3, height), 'count': 0, 'points': []},
    {'name': 'Center Zone', 'coords': (width//3, 0, 2*width//3, height), 'count': 0, 'points': []},
    {'name': 'Right Zone', 'coords': (2*width//3, 0, width, height), 'count': 0, 'points': []},
]
zone_colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0)]

def is_point_in_zone(point, zone):
    x, y = point
    x1, y1, x2, y2 = zone['coords']
    return x1 <= x <= x2 and y1 <= y <= y2

def save_to_db(timestamp, zone_name, count):
    conn = None
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO zone_counts (timestamp, zone_name, count) VALUES (%s, %s, %s)",
            (timestamp, zone_name, count)
        )
        conn.commit()
    except Exception as e:
        logging.error(f"Database error: {e}")
    finally:
        if conn:
            cursor.close()
            conn.close()

def video_processing():
    cap = cv2.VideoCapture(video_path)
    last_save_time = datetime.now()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        current_time = datetime.now()
        
        # Сброс счетчиков
        for zone in zones:
            zone['count'] = 0
            zone['points'] = []

        # Детекция объектов
        results = model(frame, classes=[0], verbose=False)
        boxes = results[0].boxes.xyxy.cpu().numpy()

        for box in boxes:
            x_center = int((box[0] + box[2]) // 2)
            y_center = int((box[1] + box[3]) // 2)
            
            for zone in zones:
                if is_point_in_zone((x_center, y_center), zone):
                    zone['count'] += 1
                    zone['points'].append((x_center, y_center))
                    break

        # Визуализация
        annotated_frame = frame.copy()
        for i, zone in enumerate(zones):
            color = zone_colors[i]
            x1, y1, x2, y2 = zone['coords']
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
            for (x, y) in zone['points']:
                cv2.circle(annotated_frame, (x, y), 5, color, -1)

        # Отображение счетчиков
        y_pos = 50
        for i, zone in enumerate(zones):
            text = f"{zone['name']}: {zone['count']}"
            cv2.putText(annotated_frame, text, (20, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, zone_colors[i], 2)
            y_pos += 40

        cv2.imshow("Analysis", annotated_frame)

        # Сохранение данных
        if (current_time - last_save_time).total_seconds() >= 1:
            for zone in zones:
                save_to_db(current_time, zone['name'], zone['count'])
            last_save_time = current_time

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Веб-интерфейс
@app.route('/')
def index():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        counts = {}
        for zone in zones:
            with conn.cursor() as cursor:
                cursor.execute('''SELECT count FROM zone_counts 
                                WHERE zone_name = %s 
                                ORDER BY timestamp DESC LIMIT 1''', 
                             (zone['name'],))
                count = cursor.fetchone()
                counts[zone['name']] = count[0] if count else 0
        conn.close()
        return render_template('index.html', zones=counts)
    except Exception as e:
        return str(e), 500

@app.route('/api/average')
def get_average():
    zone = request.args.get('zone')
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        query = '''SELECT 
                    EXTRACT(DOW FROM timestamp)::int as dow,
                    EXTRACT(HOUR FROM timestamp)::int as hour,
                    AVG(count) as avg
                   FROM zone_counts
                   WHERE zone_name = %s
                   GROUP BY dow, hour'''
        df = pd.read_sql_query(query, conn, params=(zone,))
        df['day'] = df['dow'].map({
            0: 'Sunday', 1: 'Monday', 2: 'Tuesday',
            3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday'
        })
        conn.close()
        return jsonify(df.to_dict(orient='records'))
    except Exception as e:
        return str(e), 500

@app.route('/download')
def download_data():
    start = request.args.get('start')
    end = request.args.get('end')
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        query = '''SELECT * FROM zone_counts 
                 WHERE timestamp BETWEEN %s AND %s'''
        df = pd.read_sql_query(query, conn, params=(start, end))
        conn.close()
        
        output = io.BytesIO()
        with pd.ExcelWriter(output) as writer:
            df.to_excel(writer, index=False)
        output.seek(0)
        
        return send_file(output, 
                        mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                        download_name='data.xlsx',
                        as_attachment=True)
    except Exception as e:
        return str(e), 500

if __name__ == '__main__':
    video_thread = threading.Thread(target=video_processing)
    video_thread.daemon = True
    video_thread.start()
    
    from waitress import serve
    serve(app, host="0.0.0.0", port=5000)

INFO:waitress:Serving on http://0.0.0.0:5000


In [ ]:
from ultralytics import YOLO
import cv2
import psycopg2
from datetime import datetime, timedelta
from flask import Flask, render_template, request, jsonify, send_file
import pandas as pd
import threading
import io
import logging
import xlsxwriter

# Настройка логирования
logging.basicConfig(level=logging.DEBUG)

# Инициализация Flask
app = Flask(__name__)

# Конфигурация PostgreSQL
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'dbname': 'people_counter',
    'user': 'pc_user',
    'password': 'strong_password'
}

# Константы
MAX_DAYS_PERIOD = 31
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'

# Инициализация модели YOLO
model = YOLO("yolov8n.pt")
video_path = "videoAUD.mp4"

# Инициализация видеопотока
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# Определение зон
zones = [
    {'name': 'Left Zone', 'coords': (0, 0, width//3, height), 'count': 0, 'points': []},
    {'name': 'Center Zone', 'coords': (width//3, 0, 2*width//3, height), 'count': 0, 'points': []},
    {'name': 'Right Zone', 'coords': (2*width//3, 0, width, height), 'count': 0, 'points': []},
]
zone_colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0)]

# CORS headers
@app.after_request
def add_cors_headers(response):
    response.headers['Access-Control-Allow-Origin'] = '*'
    response.headers['Access-Control-Allow-Headers'] = 'Content-Type'
    return response

def is_point_in_zone(point, zone):
    x, y = point
    x1, y1, x2, y2 = zone['coords']
    return x1 <= x <= x2 and y1 <= y <= y2

def save_to_db(timestamp, zone_name, count):
    conn = None
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO zone_counts (timestamp, zone_name, count) VALUES (%s, %s, %s)",
            (timestamp, zone_name, count)
        )
        conn.commit()
    except Exception as e:
        logging.error(f"Database error: {e}")
    finally:
        if conn:
            cursor.close()
            conn.close()

def video_processing():
    cap = cv2.VideoCapture(video_path)
    last_save_time = datetime.now()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        current_time = datetime.now()
        
        if (current_time - last_save_time).total_seconds() >= 10:
            for zone in zones:
                zone['count'] = 0
                zone['points'] = []

            results = model(frame, classes=[0], verbose=False)
            boxes = results[0].boxes.xyxy.cpu().numpy()

            for box in boxes:
                x_center = int((box[0] + box[2]) // 2)
                y_center = int((box[1] + box[3]) // 2)
                
                for zone in zones:
                    if is_point_in_zone((x_center, y_center), zone):
                        zone['count'] += 1
                        zone['points'].append((x_center, y_center))
                        break

            for zone in zones:
                save_to_db(current_time, zone['name'], zone['count'])
            last_save_time = current_time

        annotated_frame = frame.copy()
        for i, zone in enumerate(zones):
            color = zone_colors[i]
            x1, y1, x2, y2 = zone['coords']
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
            for (x, y) in zone['points']:
                cv2.circle(annotated_frame, (x, y), 5, color, -1)

        y_pos = 50
        for i, zone in enumerate(zones):
            text = f"{zone['name']}: {zone['count']}"
            cv2.putText(annotated_frame, text, (20, y_pos), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, zone_colors[i], 2)
            y_pos += 40

        cv2.imshow("Analysis", annotated_frame)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

@app.route('/')
def index():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        counts = {}
        for zone in zones:
            with conn.cursor() as cursor:
                cursor.execute('''SELECT count FROM zone_counts 
                                WHERE zone_name = %s 
                                ORDER BY timestamp DESC LIMIT 1''', 
                             (zone['name'],))
                count = cursor.fetchone()
                counts[zone['name']] = count[0] if count else 0
        conn.close()
        return render_template('index.html', zones=counts)
    except Exception as e:
        return str(e), 500

@app.route('/api/average')
def get_average():
    zone = request.args.get('zone')
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        query = '''SELECT 
                    EXTRACT(DOW FROM timestamp)::int as dow,
                    EXTRACT(HOUR FROM timestamp)::int as hour,
                    AVG(count) as avg
                   FROM zone_counts
                   WHERE zone_name = %s
                   GROUP BY dow, hour'''
        df = pd.read_sql_query(query, conn, params=(zone,))
        df['day'] = df['dow'].map({
            0: 'Sunday', 1: 'Monday', 2: 'Tuesday',
            3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday'
        })
        conn.close()
        return jsonify(df.to_dict(orient='records'))
    except Exception as e:
        return str(e), 500

@app.route('/download')
def download_data():
    # Получение и нормализация параметров
    start = request.args.get('start', '').replace('T', ' ')
    end = request.args.get('end', '').replace('T', ' ')
    format = request.args.get('format', 'xlsx')

    # Валидация параметров
    if not start or not end:
        logging.error("Missing start/end parameters")
        return "Start and end parameters are required", 400

    try:
        start_dt = datetime.strptime(start, DATE_FORMAT)
        end_dt = datetime.strptime(end, DATE_FORMAT)
    except ValueError as e:
        logging.error(f"Invalid date format: {str(e)}")
        return f"Invalid date format. Use {DATE_FORMAT}", 400

    if (end_dt - start_dt).days > MAX_DAYS_PERIOD:
        logging.error("Date range exceeded")
        return f"Max period is {MAX_DAYS_PERIOD} days", 400

    logging.info(f"Processing download request: {start} to {end}")

    conn = None
    try:
        # Подключение к БД
        conn = psycopg2.connect(**DB_CONFIG)
        query = '''
            SELECT 
                TO_CHAR(timestamp, 'YYYY-MM-DD HH24:MI:SS') as timestamp,
                zone_name,
                count
            FROM zone_counts
            WHERE timestamp BETWEEN %s AND %s
            ORDER BY timestamp DESC
        '''
        logging.debug(f"Executing query: {query} with params: {start}, {end}")
        
        # Выполнение запроса
        df = pd.read_sql_query(query, conn, params=(start, end))
        
        if df.empty:
            logging.warning("No data found for query")
            return "No data found for selected period", 404

        # Создание файла
        if format == 'csv':
            output = io.StringIO()
            df.to_csv(output, index=False)
            mimetype = 'text/csv'
            filename = 'people_counts.csv'
        else:
            output = io.BytesIO()
            with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
                df.to_excel(writer, 
                          index=False, 
                          sheet_name='People Counts',
                          header=['Timestamp', 'Zone Name', 'Count'])
                
                # Форматирование Excel
                workbook = writer.book
                worksheet = writer.sheets['People Counts']
                header_format = workbook.add_format({
                    'bold': True,
                    'text_wrap': True,
                    'border': 1
                })
                
                for col_num, value in enumerate(df.columns.values):
                    worksheet.write(0, col_num, value, header_format)
                    worksheet.set_column(col_num, col_num, 20)
                
                writer.close()
            
            mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
            filename = 'people_counts.xlsx'
        
        output.seek(0)
        logging.info("File successfully generated")
        
        return send_file(
            output,
            mimetype=mimetype,
            download_name=filename,
            as_attachment=True
        )

    except Exception as e:
        logging.error(f"Download failed: {str(e)}", exc_info=True)
        return f"Server error: {str(e)}", 500
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    # Проверка существования таблицы
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS zone_counts (
                    id SERIAL PRIMARY KEY,
                    timestamp TIMESTAMP NOT NULL,
                    zone_name VARCHAR(50) NOT NULL,
                    count INTEGER NOT NULL
                )
            ''')
            conn.commit()
        conn.close()
    except Exception as e:
        logging.error(f"Database initialization failed: {str(e)}")
        exit(1)

    # Запуск потоков
    video_thread = threading.Thread(target=video_processing)
    video_thread.daemon = True
    video_thread.start()
    
    from waitress import serve
    serve(app, host="0.0.0.0", port=5000)

INFO:waitress:Serving on http://0.0.0.0:5000
